# 제4 고지 : 신경망 만들기 
## STEP 39 : 합계 함수

이번 단계에서는 DeZero에 합계를 구하는 `sum`함수를 추가한다. 우선 이전에 구현한 **'덧셈'의 미분을 복습한 후 이를 응용하여 구현**한다.

### 39.1 sum 함수의 역전파

덧셈함수를 다시 복습하면, $y=x_0+x_1$ 일 때, $\frac{\partial y}{\partial x_0}=1, \frac{\partial y}{\partial x_1}=1$ 이므로 **역전파를 그대로 흘려 보내주기만 했으면 됐다.**

<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2039-1.png' align='center' width='50%'>
</p>

즉, **출력쪽에서 전해준 1 이라는 기울기를 단순히 복사**만 하면되는데, 사실 이는 원소가 $N$개인 벡터를 사용하더라도 다음과 같이 **기울기 벡터 갯수만큼만 복사하여 입력변수의 형상과 같아지도록 하기만 하면 된다.**
2개           |  $N$개
:-------------------------:|:-------------------------:
![](../assets/deep_learning_2_images/fig%201-18.png)  |  ![](../assets/%EA%B7%B8%EB%A6%BC%2039-3.png)
![](../assets/%EA%B7%B8%EB%A6%BC%2039-2.png)  |  ![](../assets/deep_learning_2_images/fig%201-22.png)



### 39.2 sum 함수 구현

DeZero의 `sum` 함수 역전파에서는 입력의 형상과 같아지도록 **기울기의 원소를 '복사'** 한다. 그런데 `Variable` 인스턴스를 사용하므로 복사작업도 DeZero 함수로 구현해야한다. 이를 위해 그 다음 단계 step40. 에서 살펴볼 `broadcast_to` 함수를 미리 사용한다. 해당 함수는 넘파이의 브로드캐스트와 같은 기능으로 **지정한 형상이 되도록 원소를 복사하는 함수**이다.

```python
class Sum(Function):
    def forward(self, x):
        self.x_shape=x.shape
        y = x.sum()
        return y

    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx


def sum(x):
    return Sum()(x)
```

In [1]:
import sys 
sys.path.append("..")

import numpy as np 
from dezero import Variable
import dezero.functions as F 

x = Variable(np.array([1,2,3,4,5,6]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)
print("="*10)

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)


variable(21)
variable([1 1 1 1 1 1])
variable(21)
variable([[1 1 1]
          [1 1 1]])


### 39.3 axis 와 keepdims

지금까지 `sum`함수의 기본을 완성했다. 하지만 `np.sum()` 은 **axis와 keepdims 를 이용해 더 정교하게 작동**한다.

<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2039-4.png' align='center' width='50%'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2039-5.png' align='center' width='50%'>
</p>

`axis` 는 **합계를 구하는 기준 축**으로 **int 외에도 `None`과 `tuple`도 받는다.**

- `None`이면 모든 값을 더한 스칼라값을 리턴.
- `(0,2)`와 같은 튜플이면 0번과 2번 축 모두에 대한 합계를 리턴.

그리고 `keepdims` 는 **축의 수를 유지하여 리턴**한다.




In [2]:
x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x,axis=0)
print(y)
print(x.shape,"->",y.shape)

print("="*10,"axis=None","="*10)
y = np.sum(x,axis=None)
print(y)
print(x.shape,"->",y.shape)

print("="*10,"axis=(0,1)","="*10)
y = np.sum(x,axis=(0,1))
print(y)
print(x.shape,"->",y.shape)

print("="*10,"keepdims=True","="*10)
y = np.sum(x,axis=0,keepdims=True)
print(y)
print(x.shape,"->",y.shape)

[5 7 9]
(2, 3) -> (3,)
========== axis=None ==========
21
(2, 3) -> ()
========== axis=(0,1) ==========
21
(2, 3) -> ()
========== keepdims=True ==========
[[5 7 9]]
(2, 3) -> (1, 3)


이제 이를 바탕으로 DeZero 또한 `axis` 와 `keepdims`를 받을 수 있도록 `sum` 함수를 수정 구현한다. 또한 `sum` 함수를  `Variable` 에서 사용할 수 있도록 수정한다.

```python
class Sum(Function):
    ####################################
    def __init__(self, axis, keepdims):
        self.axis = axis
        self.keepdims = keepdims
    ####################################

    def forward(self, x):
        self.x_shape = x.shape
        ####################################
        y = x.sum(axis=self.axis, keepdims=self.keepdims)
        ####################################
        return y

    def backward(self, gy):
        ####################################
        # 해당 함수는 gy의 형상을 미세하게 조정하는 함수로 넘파이와 관련된 문제이므로 따로 다루지 않는다.
        gy = utils.reshape_sum_backward(gy, self.x_shape, self.axis, self.keepdims)
        ####################################
        gx = broadcast_to(gy, self.x_shape)
        return gx

####################################
def sum(x, axis=None, keepdims=False):
    return Sum(axis, keepdims)(x)
####################################
```

```python
class Variable:
    ...
    
    def sum(self, axis=None, keepdims=False):
        return dezero.functions.sum(self, axis, keepdims)
```

In [8]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x,axis=0)
y.backward()
print(y)
print(x.grad)

print("="*10)

x = Variable(np.random.randn(2,3,4,5))
y = x.sum(keepdims=True)
print(y)
print(y.shape)

variable([5 7 9])
variable([[1 1 1]
          [1 1 1]])
variable([[[[10.31125735]]]])
(1, 1, 1, 1)
